In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
path = "/content/drive/MyDrive/TFM/"
path1 = path + "Planets.csv"
path2 = path + "Not_Planets.csv"

In [3]:
#Creación del dataset con los registros de secuencias positivas y secuencias negativas
planets_df = pd.read_csv(path1, names=list(range(4000)))
not_planets_df = pd.read_csv(path2, names=list(range(4000)))
X_data = pd.concat([planets_df, not_planets_df]).to_numpy()
y_data = np.concatenate((np.ones(len(planets_df)), np.zeros(len(not_planets_df))))

#Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)
y_train = np.transpose(np.atleast_2d(y_train))
y_test = np.transpose(np.atleast_2d(y_test))

In [7]:
#Definición del modelo a entrenar
from tensorflow.keras import layers
from tensorflow import keras

model_name = "Conv2D_D"

model = keras.Sequential(
    [
        layers.Input(shape=(4000,1)),
        layers.Reshape((500,8,1)),
        layers.Conv2D(filters=32, kernel_size=(5,1), activation='relu'),
        layers.MaxPooling2D(pool_size=(3,1)),
        layers.Dropout(0.2),
        layers.Conv2D(filters=32, kernel_size=(5,1), activation='relu'),
        layers.MaxPooling2D(pool_size=(3,1)),
        layers.Dropout(0.2),
        layers.Conv2D(filters=32, kernel_size=(5,1), activation='relu'),
        layers.MaxPooling2D(pool_size=(3,1)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(100, activation='sigmoid'),
        layers.Dense(10, activation='sigmoid'),
        layers.Dense(1, activation='sigmoid')
    ],
    name=model_name
)

In [8]:
from datetime import datetime
#Compilación del modelo
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
              keras.metrics.BinaryAccuracy(),
              keras.metrics.Recall(),
              keras.metrics.Precision(),
              keras.metrics.AUC(num_thresholds=25)
            ]
    )

#Definición del directorio donde guardar los datos del entrenamiento
logdir = "/content/drive/MyDrive/TFM/Logs/" + model.name + "/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/TFM/Models/" + model_name + "/best_model.keras",
    verbose=1,
    save_best_only=True,
    mode="max",
    monitor="val_binary_accuracy",
)

In [9]:
training_history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback, model_checkpoint],
)

Epoch 1/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - auc_1: 0.5773 - binary_accuracy: 0.5666 - loss: 0.6711 - precision_1: 0.5432 - recall_1: 0.7376
Epoch 1: val_binary_accuracy improved from -inf to 0.59800, saving model to /content/drive/MyDrive/TFM/Models/Conv2D_D/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - auc_1: 0.5774 - binary_accuracy: 0.5666 - loss: 0.6711 - precision_1: 0.5432 - recall_1: 0.7378 - val_auc_1: 0.5790 - val_binary_accuracy: 0.5980 - val_loss: 0.6489 - val_precision_1: 0.5685 - val_recall_1: 0.9893
Epoch 2/50
515/520 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - auc_1: 0.5821 - binary_accuracy: 0.5848 - loss: 0.6537 - precision_1: 0.5552 - recall_1: 0.9808
Epoch 2: val_binary_accuracy improved from 0.59800 to 0.60438, saving model to /content/drive/MyDrive/TFM/Models/Conv2D_D/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - auc_1: 0.5822 - binary_accuracy: 0.5849 - loss: 0.6536 - precision_1: 0.5553 - recall_1: 0.9808 - val_auc_1: 0.5872 - val